In [1]:
# ====================================================================================
# Multi-periodusu portfolio optimalizalas - Python Verzio V1.0
#
# Szerzo: Vereszki Péter
# Datum: 2025-10-07
#
# Ez a szkript az R-ben kifejlesztett es kidebuggolt V45.6-os verzio
# teljes, funkcionalisan azonos Python portja.
# - Hasznalt technologiak: PyTorch, Pandas, Numpy, Scipy, Plotly
# - GPU es JIT optimalizalt a maximalis teljesitmenyert.
# - Teljesen kompatibilis az R verzio altal generalt HDF5 cache fajlokkal.
# ====================================================================================


# ====================================================================================
# 0. LEPES: KORNYEZET BEALLITASA ES CSOMAGOK TELEPITESE
#
# Ezt a cellat egy Kaggle vagy Colab notebook elejen kell futtatni.
# ====================================================================================
try:
    import qpsolvers
    import yfinance
    import kaleido
    import plotly
    import cvxopt
    from cvxopt import matrix
    import cvxpy # Hozzáadva az ellenőrzéshez
    import ecos  # Hozzáadva az ellenőrzéshez
    print(">>> Szukseges csomagok mar telepitve.")
except ImportError:
    print(">>> Szukseges csomagok telepitese...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", 
                           "yfinance", "qpsolvers", "plotly>=5.15.0", "kaleido==0.2.1", "cvxopt",
                           "cvxpy", "ecos",
                           "--quiet"])
    print(">>> Telepites befejezve.")


# ====================================================================================
# 1. LEPES: IMPORTALAS ES GLOBALIS BEALLITASOK
# ====================================================================================
import os
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
import pickle
import h5py

import torch
import torch.nn.functional as F
from scipy.optimize import linprog
import cvxopt
from cvxopt import matrix
from qpsolvers import solve_qp

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Globalis eszkoz beallitasa (GPU, ha elerheto)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f">>> Hasznalt eszkoz: {DEVICE}")

# Reprodukalhatosag
torch.manual_seed(42)
np.random.seed(42)


# ====================================================================================
# 2. LEPES: KONFIGURACIOS PARAMETEREK
# ====================================================================================
config = {
    # Modell futasi parameterek
    "num_months": 60,
    "num_scenarios": 10000,
    "expected_block_length": 60,
    
    # Hibrid cel-lekerdezes parameterei
    "alpha": 0.5,
    "gamma": 0.995,
    
    # Optimalizalo parameterek
    "epochs": 10000,
    "lr_initial": 0.01,
    "lr_step_size": 4000,
    "lr_gamma": 0.2,
    
    # Strategiai kenyszerek
    "min_turnover": 0.02,
    "max_turnover": 0.03,
    "monthly_risk_min": 0.0284,
    "monthly_risk_max": 0.0404,
    
    # Bunteto lambdak
    "lambda_turnover": 1000.0,
    "lambda_monthly_risk": 5000000.0,
    "lambda_wealth": 10000.0,
    "lambda_anchor": 100.0,
    
    # Fajlnevek es eleresi utak
    "scenario_cache": "scenarios_cache.h5",
    "output_dir": "outputs_py",
    "output_prefix": "point_py"
}

# ====================================================================================
# 3. LEPES: FUGGVENY DEFINICIOK
# ====================================================================================

def load_prepared_data_from_csv(data_path: str, meta_path: str) -> tuple[pd.DataFrame, np.ndarray]:
    """
    Betölti az adatelőkészítő notebook által generált meta- és adatfájlokat.
    Különválasztja az eszközhozamokat a makro adatoktól (bár utóbbit most nem használjuk).
    Visszaadja az ETF metaadatokat és a historikus hozamok NumPy tömbjét.
    """
    print(">>> Adatok betöltése a központosított CSV forrásból...")
    
    # Metaadatok betöltése
    meta_df_full = pd.read_csv(meta_path)
    
    # Csak az eszközhozamokhoz tartozó metaadatok szűrése
    etf_meta_df = meta_df_full[meta_df_full['type'] == 'asset_return'].copy()
    etf_symbols = etf_meta_df['symbol'].tolist()
    
    # Teljes adatsor betöltése
    data_df = pd.read_csv(data_path, index_col='Date', parse_dates=True)
    
    # Csak a releváns eszközhozam oszlopok kiválasztása, a metaadatok szerinti sorrendben
    # Ez kulcsfontosságú a konzisztencia szempontjából!
    returns_df = data_df[etf_symbols]
    
    # Átalakítás NumPy tömbbé
    historical_returns_matrix = returns_df.values.astype(np.float32)
    
    print(f"Betöltve {historical_returns_matrix.shape[0]} hónapnyi adat {len(etf_symbols)} eszközre.")
    
    return etf_meta_df, historical_returns_matrix


def generate_bootstrap_scenario_py(historical_data, num_months, expected_block_length):
    """Egyetlen stacionarius geometriai bootstrap szcenariot general."""
    n_hist, n_assets = historical_data.shape
    p = 1 / expected_block_length
    scenario = np.zeros((num_months, n_assets))
    current_idx = np.random.randint(0, n_hist)
    scenario[0, :] = historical_data[current_idx, :]
    
    for i in range(1, num_months):
        if np.random.rand() < p:
            current_idx = np.random.randint(0, n_hist)
        else:
            current_idx = (current_idx + 1) % n_hist
        scenario[i, :] = historical_data[current_idx, :]
    return scenario

# HELYETTESÍTSD EZT A FÜGGVÉNYT A MEGLÉVŐVEL

def generate_or_load_scenarios_py(historical_data, cfg, investable_symbols: list):
    """
    Szcenariok generalasa vagy betoltese.
    VÉGLEGES, ROBUSZTUS VERZIÓ: A HDF5 fájlból betöltött adatokat a dimenziók mérete
    alapján azonosítja és a helyes (idő, eszköz, szcenárió) sorrendbe transzponálja,
    majd leszűri a befektethető eszközökre.
    """
    cache_file = cfg['scenario_cache']
    
    if os.path.exists(cache_file):
        print(f"Toltott szcenariok a(z) '{cache_file}' cache-bol...")
        with h5py.File(cache_file, 'r') as f:
            # 1. Alapvető metaadatok beolvasása
            scenarios_full_raw = f['scenarios'][:]
            all_symbols_from_h5 = [s.decode('utf-8') for s in f['asset_symbols'][:]]
            probabilities = f['posterior_probabilities'][:]
            
            # 2. Dimenziók méretének meghatározása a megbízható forrásokból
            expected_months = cfg['num_months']
            expected_assets_total = len(all_symbols_from_h5)
            expected_scenarios = len(probabilities)
            
            print(f"Várt dimenzióméretek: Idő={expected_months}, Eszközök={expected_assets_total}, Szcenáriók={expected_scenarios}")
            print(f"Beolvasott tömb mérete: {scenarios_full_raw.shape}")

            # 3. Dimenziók indexének dinamikus azonosítása
            try:
                shape_list = list(scenarios_full_raw.shape)
                time_dim_idx = shape_list.index(expected_months)
                asset_dim_idx = shape_list.index(expected_assets_total)
                scenario_dim_idx = shape_list.index(expected_scenarios)
            except ValueError:
                raise ValueError(f"A beolvasott tömb mérete {scenarios_full_raw.shape} nem egyezik a várt méretekkel!")

            # 4. Helyes transzponálás végrehajtása a (idő, eszköz, szcenárió) célformátumra
            scenarios_full_transposed = np.transpose(scenarios_full_raw, (time_dim_idx, asset_dim_idx, scenario_dim_idx))
            print(f"Helyes sorrendbe transzponált teljes tömb mérete: {scenarios_full_transposed.shape}")

            # 5. Befektethető eszközök indexeinek megkeresése
            investable_indices = [all_symbols_from_h5.index(s) for s in investable_symbols]
            
            # 6. Szeletelés: a teljes tömbből kiválasztjuk a befektethetőket
            scenarios_array = scenarios_full_transposed[:, investable_indices, :]
            print(f"Szűrés utáni szcenárió-tömb mérete: {scenarios_array.shape}")

            # 7. Ellenőrzés (most már helyes lesz)
            num_scenarios = scenarios_array.shape[2]
            assert len(probabilities) == num_scenarios, f"Valószínűségek ({len(probabilities)}) és szcenáriók ({num_scenarios}) száma nem egyezik a szűrés után!"

    else:
        # A generálás rész változatlan
        print(f"Szcenariok generalasa... ({cfg['scenario_cache']} nem talalhato)")
        scenarios_list = [generate_bootstrap_scenario_py(historical_data, cfg['num_months'], cfg['expected_block_length']) for _ in range(cfg['num_scenarios'])]
        scenarios_array = np.stack(scenarios_list, axis=-1)
        probabilities = np.full(cfg['num_scenarios'], 1.0 / cfg['num_scenarios'], dtype=np.float32)
    
    return scenarios_array, probabilities
    
### Kényszerrendszer építése DataFrame-ből
def build_constraints_from_dataframe(etf_symbols: list, constraints_spec: dict) -> dict:
    """
    Felépíti a kényszerrendszert a pandas DataFrame deklaratív definíciója alapján.
    Minden releváns kényszert tartalmaz az RL projektből.
    A kényszerek >= formában vannak.
    """
    n_assets = len(etf_symbols)
    df = pd.DataFrame(columns=etf_symbols + ['type', 'rhs'])

    # === ALAPVETŐ KÉNYSZEREK (NEM VÁLTOZNAK AZ IDŐBEN) ===
    # sum(w) = 1
    df.loc['sum_w_eq_1'] = [1.0] * n_assets + ['eq', 1.0]
    # w_i >= 0
    for i, symbol in enumerate(etf_symbols):
        row = [0.0] * n_assets + ['ineq', 0.0]; row[i] = 1.0
        df.loc[f'{symbol}_gte_0'] = row
    # w_i <= 0.4  =>  -w_i >= -0.4
    for i, symbol in enumerate(etf_symbols):
        row = [0.0] * n_assets + ['ineq', -0.4]; row[i] = -1.0
        df.loc[f'{symbol}_lte_04'] = row
        
    # === STRATÉGIAI MANDÁTUMOK (IDŐBEN VÁLTOZNAK) ===
    # BIL + SHY >= min_bil_shy
    bil_shy_tickers = ["BIL", "SHY"]
    df.loc['bil_shy_gte_spec'] = [1.0 if s in bil_shy_tickers else 0.0 for s in etf_symbols] + ['ineq', constraints_spec['min_bil_shy']]
    
    # Treasury >= min_treasury
    treasury_tickers = ["BIL", "SHY", "IEF", "TLT"]
    df.loc['treasury_gte_spec'] = [1.0 if s in treasury_tickers else 0.0 for s in etf_symbols] + ['ineq', constraints_spec['min_treasury']]

    # GLD >= min_gld
    df.loc['gld_gte_spec'] = [1.0 if s == "GLD" else 0.0 for s in etf_symbols] + ['ineq', constraints_spec['min_gld']]
    
    # GLD <= max_gld  =>  -GLD >= -max_gld
    df.loc['gld_lte_spec'] = [-1.0 if s == "GLD" else 0.0 for s in etf_symbols] + ['ineq', -constraints_spec['max_gld']]

    # Mátrixok szétválasztása
    A_eq = df[df['type'] == 'eq'][etf_symbols].values.astype(np.float64)
    b_eq = df[df['type'] == 'eq']['rhs'].values.astype(np.float64)
    A_ineq = df[df['type'] == 'ineq'][etf_symbols].values.astype(np.float64)
    b_ineq = df[df['type'] == 'ineq']['rhs'].values.astype(np.float64)

    return {"A_eq": A_eq, "b_eq": b_eq, "A_ineq": A_ineq, "b_ineq": b_ineq}


# ====================================================================================
# 3. LEPES: FUGGVENY DEFINICIOK
# ====================================================================================
import cvxpy as cp

import torch

# ====================================================================================
# 1. LÉPÉS: SÚLYOZOTT KVANTILIS SEGÉDFÜGGVÉNY
# Ez a függvény szükséges a súlyozott VaR számításokhoz.
# ====================================================================================

def weighted_quantile(values: torch.Tensor, quantiles: torch.Tensor, weights: torch.Tensor, dim: int = -1) -> torch.Tensor:
    """
    Robusztus, differenciálható és GPU-kompatibilis súlyozott kvantilis számítás.
    A `torch.quantile`-hoz hasonlóan működik, de figyelembe veszi a súlyokat.

    Args:
        values: A tenzor, amelynek a kvantilisét keressük.
        quantiles: A keresett kvantilis(ek) (pl. 0.05).
        weights: A 'values' elemeihez tartozó valószínűségi súlyok.
        dim: A dimenzió, amely mentén a számítást végezzük.

    Returns:
        A súlyozott kvantilis(ek).
    """
    # Győződjünk meg róla, hogy a súlyok összege 1
    if not torch.isclose(torch.sum(weights), torch.tensor(1.0)):
        weights = weights / torch.sum(weights)

    # Értékek és súlyok rendezése
    sorted_indices = torch.argsort(values, dim=dim)
    sorted_values = torch.gather(values, dim, sorted_indices)
    sorted_weights = torch.gather(weights.expand_as(values), dim, sorted_indices)

    # Kumulatív súlyok számítása
    cum_weights = torch.cumsum(sorted_weights, dim=dim)

    # A kvantilisek helyének megtalálása a kumulatív súlyok között
    # A lineáris interpolációhoz megtaláljuk a két szomszédos pontot
    target_indices = torch.searchsorted(cum_weights, quantiles)

    # Szélsőértékek kezelése, hogy ne fussunk ki az indexből
    target_indices.clamp_(0, sorted_values.shape[dim] - 1)

    # Súlyok a lineáris interpolációhoz
    # Az alsó határ súlya
    weight_lower = (cum_weights.gather(dim, target_indices) - quantiles) / \
                   sorted_weights.gather(dim, target_indices).clamp(min=1e-12)
    weight_lower.clamp_(0, 1)

    # Az alsó határ indexe
    idx_lower = (target_indices - 1).clamp(min=0)

    # Az interpolált érték kiszámítása
    result = sorted_values.gather(dim, idx_lower) * weight_lower + \
             sorted_values.gather(dim, target_indices) * (1 - weight_lower)

    return result.squeeze(dim)


# ====================================================================================
# 2. LÉPÉS: A HIBRID VESZTESÉGFÜGGVÉNY TELJES, SÚLYOZOTT VERZIÓJA
# ====================================================================================

def calculate_hybrid_loss(w_matrix, scenarios_tensor, probabilities_t, 
                          target_wealth_t, turnover_limits_t, tx_cost_vec_t, 
                          gamma_discounts_t, return_components: bool = False):
    """
    Kiszámolja a hibrid veszteséget a megadott portfóliósúly-pályára.
    MODOSÍTVA: A posterior valószínűségeket használja a statisztikák számításához,
    így képes kezelni az Entrópia Poolingból származó nem-egyenletes szcenárió-súlyokat.
    """
    q_level_tensor = torch.tensor(0.05, device=DEVICE)
    epsilon = 1e-12 # Numerikus stabilitáshoz

    # A Softmax már korábban megtörtént, a w_matrix a végső eszközsúly
    w_matrix_lagged = torch.cat([w_matrix[0:1, :], w_matrix[:-1, :]], dim=0)
    avg_turnover = torch.mean(0.5 * torch.sum(torch.abs(w_matrix - w_matrix_lagged), dim=1))
    monthly_tx_cost = 0.5 * torch.sum(torch.abs(w_matrix - w_matrix_lagged) * tx_cost_vec_t, dim=1)
    
    net_monthly_returns = torch.einsum("mas,ma->ms", scenarios_tensor, w_matrix) - monthly_tx_cost.unsqueeze(1)
    terminal_wealths = torch.prod(1 + net_monthly_returns, dim=0)
    
    # --- TERMINÁLIS KOCKÁZATOK (SÚLYOZOTT VÁLTOZAT) ---
    expected_terminal_wealth = torch.sum(terminal_wealths * probabilities_t)
    var_threshold = weighted_quantile(terminal_wealths, q_level_tensor, probabilities_t)
    
    # A farok-szcenáriók és valószínűségeik azonosítása
    tail_mask = terminal_wealths < var_threshold
    tail_values = terminal_wealths[tail_mask]
    tail_probabilities = probabilities_t[tail_mask]
    
    # Súlyozott átlag a farokban (CVaR) - a farok-valószínűségek normalizálásával
    sum_tail_probs = torch.sum(tail_probabilities)
    terminal_cvar_raw = torch.sum(tail_values * tail_probabilities) / (sum_tail_probs + epsilon)
    
    terminal_cCVaR = expected_terminal_wealth - terminal_cvar_raw
    
    # --- HAVI KOCKÁZATOK (SÚLYOZOTT VÁLTOZAT) ---
    # Súlyozott havi várható hozam
    exp_ret_monthly = torch.sum(net_monthly_returns * probabilities_t.unsqueeze(0), dim=1, keepdim=True)
    
    # Súlyozott havi VaR (minden hónapra külön)
    var_thr_monthly = torch.stack([
        weighted_quantile(net_monthly_returns[t], q_level_tensor, probabilities_t)
        for t in range(net_monthly_returns.shape[0])
    ]).unsqueeze(1)
    
    # Súlyozott havi CVaR (tenzoros műveletekkel a hatékonyságért)
    monthly_tail_mask = net_monthly_returns < var_thr_monthly
    
    # Csak a farokhozamokat és -valószínűségeket tartjuk meg
    tail_returns_monthly = torch.where(monthly_tail_mask, net_monthly_returns, 0.0)
    tail_probs_monthly = torch.where(monthly_tail_mask, probabilities_t.unsqueeze(0), 0.0)
    
    # Súlyozott összeg a farokban, minden hónapra
    sum_weighted_tail_returns = torch.sum(tail_returns_monthly * probabilities_t.unsqueeze(0), dim=1, keepdim=True)
    
    # A farok valószínűségeinek összege, minden hónapra
    sum_monthly_tail_probs = torch.sum(tail_probs_monthly, dim=1, keepdim=True)
    
    cvar_raw_monthly = sum_weighted_tail_returns / (sum_monthly_tail_probs + epsilon)
    monthly_cCVaR = -1 * (cvar_raw_monthly - exp_ret_monthly)

    # --- VESZTESÉGEK ÉS BÜNTETÉSEK ÖSSZEÁLLÍTÁSA (VÁLTOZATLAN LOGIKA) ---
    loss_risk_upper = torch.clamp(monthly_cCVaR - config['monthly_risk_max'], min=0)**2
    loss_risk_lower = torch.clamp(config['monthly_risk_min'] - monthly_cCVaR, min=0)**2
    discounted_monthly_risk_penalty = torch.mean((loss_risk_upper + loss_risk_lower) * gamma_discounts_t)
    
    loss_objective_terminal = terminal_cCVaR
    loss_objective_interim = config['lambda_monthly_risk'] * discounted_monthly_risk_penalty
    loss_objective_hybrid = (1 - config['alpha']) * loss_objective_terminal + config['alpha'] * loss_objective_interim

    # Kényszerek (csak a nem-lineárisak maradtak)
    loss_wealth_constraint = config['lambda_wealth'] * torch.clamp(target_wealth_t - expected_terminal_wealth, min=0)**2
    loss_turnover_constraint = config['lambda_turnover'] * (torch.clamp(avg_turnover - turnover_limits_t[1], min=0)**2 + torch.clamp(turnover_limits_t[0] - avg_turnover, min=0)**2)
    
    total_loss = loss_objective_hybrid + loss_wealth_constraint + loss_turnover_constraint

    if return_components:
        return {
            "total_loss": total_loss.item(),
            "Cel_Terminalis": ((1 - config['alpha']) * loss_objective_terminal).item(),
            "Cel_Interim_Palyakock": (config['alpha'] * loss_objective_interim).item(),
            "Bunt_Forgas": loss_turnover_constraint.item(),
            "Bunt_Vagyon": loss_wealth_constraint.item(),
        }
    return total_loss
    
def generate_cvar_frontier_portfolios_py(historical_returns, constraint_system, num_points=15, cvar_alpha=0.05):
    """
    Legenerál egy CVaR-E[R] hatékony frontot a megadott lineáris kényszerek mellett.
    Ez egy KONVEX optimalizálási probléma, LP-ként megoldható.
    """
    print(f"\n--- CVaR Hatékony Front generálása ({num_points} ponttal) ---")
    n_scenarios, n_assets = historical_returns.shape
    mu = np.mean(historical_returns, axis=0)

    A_eq = constraint_system.get('A_eq')
    b_eq = constraint_system.get('b_eq')
    A_ineq = constraint_system.get('A_ineq')
    b_ineq = constraint_system.get('b_ineq')

    # 1. LÉPÉS: A lehetséges hozam-tartomány meghatározása
    w_max_ret_var = cp.Variable(n_assets)
    prob_max_ret = cp.Problem(cp.Maximize(mu @ w_max_ret_var),
                              [A_eq @ w_max_ret_var == b_eq, A_ineq @ w_max_ret_var >= b_ineq])
    prob_max_ret.solve(solver=cp.ECOS)
    if prob_max_ret.status not in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        raise RuntimeError("Hiba: A maximális hozamú portfólió nem található.")
    max_return = prob_max_ret.value

    w_min_cvar_var = cp.Variable(n_assets)
    gamma_min = cp.Variable()
    z_min = cp.Variable(n_scenarios)
    objective_min = cp.Minimize(gamma_min + (1 / (cvar_alpha * n_scenarios)) * cp.sum(z_min))
    constraints_min = [ z_min >= 0, z_min >= -historical_returns @ w_min_cvar_var - gamma_min,
                        A_eq @ w_min_cvar_var == b_eq, A_ineq @ w_min_cvar_var >= b_ineq ]
    prob_min_cvar = cp.Problem(objective_min, constraints_min)
    prob_min_cvar.solve(solver=cp.ECOS)
    if prob_min_cvar.status not in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
        raise RuntimeError("Hiba: A minimális CVaR portfólió nem található.")
    min_return_at_min_cvar = mu @ w_min_cvar_var.value
    
    print(f"Front hozam tartománya: [{min_return_at_min_cvar:.4f}, {max_return:.4f}] (havi hozam)")

    # 2. LÉPÉS: A front pontjainak végigszámolása
    target_returns = np.linspace(min_return_at_min_cvar, max_return, num_points)
    basis_portfolios = []
    w, gamma, z = cp.Variable(n_assets), cp.Variable(), cp.Variable(n_scenarios)
    
    for target_ret in target_returns:
        print(f"Számítás... Célhozam: {target_ret:.4f}", end="\r")
        objective = cp.Minimize(gamma + (1 / (cvar_alpha * n_scenarios)) * cp.sum(z))
        constraints = [ z >= 0, z >= -historical_returns @ w - gamma, A_eq @ w == b_eq, 
                        A_ineq @ w >= b_ineq, mu @ w >= target_ret ]
        prob = cp.Problem(objective, constraints)
        prob.solve(solver=cp.ECOS)
        if prob.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
            basis_portfolios.append(w.value)
        elif basis_portfolios:
            basis_portfolios.append(basis_portfolios[-1])

    print("\n>>> A bázisportfóliók generálása befejeződött.               ")
    return np.array(basis_portfolios)


def old_calculate_hybrid_loss(w_matrix, scenarios_tensor, target_wealth_t, 
                          turnover_limits_t, tx_cost_vec_t, gamma_discounts_t,
                          return_components: bool = False):
    """
    Kiszamolja a hibrid veszteseget.
    MODOSITVA: Már a kész, megengedett súlymátrixot kapja (w_matrix), nem a raw_weights-t.
    Eltávolítva a hard constraint és anchor büntetések, mert az architektúra garantálja őket.
    """
    q_level_tensor = torch.tensor(0.05, device=DEVICE)
    # A Softmax már korábban megtörtént, a w_matrix a végső eszközsúly
    
    w_matrix_lagged = torch.cat([w_matrix[0:1, :], w_matrix[:-1, :]], dim=0)
    avg_turnover = torch.mean(0.5 * torch.sum(torch.abs(w_matrix - w_matrix_lagged), dim=1))
    monthly_tx_cost = 0.5 * torch.sum(torch.abs(w_matrix - w_matrix_lagged) * tx_cost_vec_t, dim=1)
    
    # ... A többi számítás (net_monthly_returns, terminal_wealths, cCVaR-ok) VÁLTOZATLAN ...
    net_monthly_returns = torch.einsum("mas,ma->ms", scenarios_tensor, w_matrix) - monthly_tx_cost.unsqueeze(1)
    terminal_wealths = torch.prod(1 + net_monthly_returns, dim=0)
    expected_terminal_wealth = torch.mean(terminal_wealths)
    var_threshold = torch.quantile(terminal_wealths, q_level_tensor)
    terminal_cvar_raw = torch.mean(terminal_wealths[terminal_wealths < var_threshold])
    terminal_cCVaR = expected_terminal_wealth - terminal_cvar_raw
    exp_ret_monthly = torch.mean(net_monthly_returns, dim=1, keepdim=True)
    var_thr_monthly = torch.quantile(net_monthly_returns, q=q_level_tensor.item(), dim=1, keepdim=True)
    mask = net_monthly_returns < var_thr_monthly
    masked_ret = torch.where(mask, net_monthly_returns, torch.tensor(float('nan'), device=DEVICE))
    cvar_raw_monthly = torch.nansum(masked_ret, dim=1, keepdim=True) / torch.sum(~torch.isnan(masked_ret), dim=1, keepdim=True)
    cvar_raw_monthly = torch.nan_to_num(cvar_raw_monthly, nan=0.0)
    monthly_cCVaR = -1 * (cvar_raw_monthly - exp_ret_monthly)
    loss_risk_upper = torch.clamp(monthly_cCVaR - config['monthly_risk_max'], min=0)**2
    loss_risk_lower = torch.clamp(config['monthly_risk_min'] - monthly_cCVaR, min=0)**2
    discounted_monthly_risk_penalty = torch.mean((loss_risk_upper + loss_risk_lower) * gamma_discounts_t)
    loss_objective_terminal = terminal_cCVaR
    loss_objective_interim = config['lambda_monthly_risk'] * discounted_monthly_risk_penalty
    loss_objective_hybrid = (1 - config['alpha']) * loss_objective_terminal + config['alpha'] * loss_objective_interim
    # ... VÁLTOZATLAN RÉSZ VÉGE ...

    # Kenyszerek (CSAK a nem-lineárisak maradtak!)
    loss_wealth_constraint = config['lambda_wealth'] * torch.clamp(target_wealth_t - expected_terminal_wealth, min=0)**2
    loss_turnover_constraint = config['lambda_turnover'] * (torch.clamp(avg_turnover - turnover_limits_t[1], min=0)**2 + torch.clamp(turnover_limits_t[0] - avg_turnover, min=0)**2)
    
    # ELTÁVOLÍTVA: loss_anchor, loss_hard_constraints
    total_loss = loss_objective_hybrid + loss_wealth_constraint + loss_turnover_constraint

    if return_components:
        return {
            "total_loss": total_loss.item(),
            "Cel_Terminalis": ((1 - config['alpha']) * loss_objective_terminal).item(),
            "Cel_Interim_Palyakock": (config['alpha'] * loss_objective_interim).item(),
            "Bunt_Forgas": loss_turnover_constraint.item(),
            "Bunt_Vagyon": loss_wealth_constraint.item(),
        }
    return total_loss


def run_single_optimization_py(target_wealth, basis_strict_t, basis_laza_t, basis_nagyon_laza_t, etf_meta, 
                               scenarios_tensor, probabilities, cfg): # <-- ÚJ 'probabilities' PARAMÉTER
    """
    Egyetlen pont optimalizalasa a hatekony fronton, JIT forditassal.
    MODOSITVA: Átveszi és felhasználja a szcenáriókhoz tartozó valószínűségeket.
    """
    num_months, num_assets = cfg['num_months'], len(etf_meta)
    num_basis_portfolios = basis_strict_t.shape[0]

    raw_weights = torch.zeros((num_months, num_basis_portfolios), device=DEVICE).requires_grad_(True)
    
    # Tenzorok elokeszitese
    target_wealth_t = torch.tensor(target_wealth, dtype=torch.float32, device=DEVICE)
    turnover_limits_t = torch.tensor([cfg['min_turnover'], cfg['max_turnover']], dtype=torch.float32, device=DEVICE)
    tx_cost_vec_t = torch.tensor(etf_meta['tx_cost'].values, dtype=torch.float32, device=DEVICE).unsqueeze(0)
    gamma_discounts_t = torch.tensor([cfg['gamma']**i for i in range(1, num_months + 1)], dtype=torch.float32, device=DEVICE).unsqueeze(1)
    
    # --- MÓDOSÍTÁS: Valószínűségi tenzor előkészítése ---
    probabilities_t = torch.tensor(probabilities, dtype=torch.float32, device=DEVICE)
        
    optimizer = torch.optim.Adam([raw_weights], lr=cfg['lr_initial'])
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=cfg['lr_step_size'], gamma=cfg['lr_gamma'])
    
    # A JIT-hez a wrapper függvény most már a külső scope-ból fogja venni a probabilities_t-t (closure)
    def loss_wrapper_for_jit(current_raw_weights):
        mixing_weights = F.softmax(current_raw_weights, dim=1)
        
        w_matrix = torch.zeros(num_months, num_assets, device=DEVICE)
        w_matrix[0:24, :]  = mixing_weights[0:24, :]  @ basis_strict_t
        w_matrix[24:48, :] = mixing_weights[24:48, :] @ basis_laza_t
        w_matrix[48:60, :] = mixing_weights[48:60, :] @ basis_nagyon_laza_t
        
        # --- MÓDOSÍTÁS: A valószínűségi tenzor átadása ---
        return calculate_hybrid_loss(w_matrix, scenarios_tensor, probabilities_t, 
                                     target_wealth_t, turnover_limits_t, tx_cost_vec_t, gamma_discounts_t)
    
    traced_loss_fn = torch.jit.trace(loss_wrapper_for_jit, (raw_weights,))

    loss_history = []
    for epoch in range(1, cfg['epochs'] + 1):
        optimizer.zero_grad()
        total_loss = traced_loss_fn(raw_weights)
        total_loss.backward()
        optimizer.step()
        scheduler.step()

        if epoch % 500 == 0:
            with torch.no_grad():
                mixing_weights = F.softmax(raw_weights, dim=1)
                final_weights_t = torch.zeros(num_months, num_assets, device=DEVICE)
                final_weights_t[0:24, :]  = mixing_weights[0:24, :]  @ basis_strict_t
                final_weights_t[24:48, :] = mixing_weights[24:48, :] @ basis_laza_t
                final_weights_t[48:60, :] = mixing_weights[48:60, :] @ basis_nagyon_laza_t
                
                # --- MÓDOSÍTÁS: A valószínűségi tenzor átadása a logoláshoz ---
                loss_components = calculate_hybrid_loss(final_weights_t, scenarios_tensor, probabilities_t, 
                                                        target_wealth_t, turnover_limits_t, tx_cost_vec_t, 
                                                        gamma_discounts_t, return_components=True)
            loss_components['Epoch'] = epoch
            loss_history.append(loss_components)
            print(f"Epoch {epoch}/{cfg['epochs']}, Loss: {loss_components['total_loss']:.4f}, LR: {scheduler.get_last_lr()[0]:.5f}")

    # Eredmények kinyerése
    with torch.no_grad():
        mixing_weights = F.softmax(raw_weights, dim=1)
        final_weights_t = torch.zeros(num_months, num_assets, device=DEVICE)
        final_weights_t[0:24, :]  = mixing_weights[0:24, :]  @ basis_strict_t
        final_weights_t[24:48, :] = mixing_weights[24:48, :] @ basis_laza_t
        final_weights_t[48:60, :] = mixing_weights[48:60, :] @ basis_nagyon_laza_t
        
        # --- MÓDOSÍTÁS: Végleges metrikák számítása a SÚLYOZOTT logikával ---
        monthly_tx_cost = 0.5 * torch.sum(torch.abs(final_weights_t - torch.cat([final_weights_t[0:1, :], final_weights_t[:-1, :]], dim=0)) * tx_cost_vec_t, dim=1)
        net_monthly_returns = torch.einsum("mas,ma->ms", scenarios_tensor, final_weights_t) - monthly_tx_cost.unsqueeze(1)
        terminal_wealths = torch.prod(1 + net_monthly_returns, dim=0)
        
        # Súlyozott várható vagyon
        expected_wealth = torch.sum(terminal_wealths * probabilities_t).item()
        
        # Súlyozott C-CVaR
        var_th = weighted_quantile(terminal_wealths, torch.tensor(0.05, device=DEVICE), probabilities_t).item()
        tail_mask = terminal_wealths < var_th
        tail_probs_normalized = probabilities_t[tail_mask] / torch.sum(probabilities_t[tail_mask])
        cvar_raw = torch.sum(terminal_wealths[tail_mask] * tail_probs_normalized)
        cCVaR_final = expected_wealth - cvar_raw.item()

    return {
        "final_weights": final_weights_t.cpu().numpy(),
        "achieved_wealth": expected_wealth,
        "terminal_cCVaR": cCVaR_final,
        "loss_history": pd.DataFrame(loss_history)
    }


# ====================================================================================
# 3. LEPES: FUGGVENY DEFINICIOK
# ====================================================================================

def generate_diagnostics_py(all_portfolios, etf_meta, scenarios_tensor, probabilities, cfg, 
                            constraints_strict, constraints_laza, constraints_nagyon_laza):
    """
    Legenerálja a diagnosztikai táblázatot és a vizualizációkat Plotly segítségével.
    MODOSÍTVA: Képes kezelni az időben változó kényszerrendszert, és
    a szcenárió-alapú metrikákat (pl. havi cCVaR) a posterior valószínűségekkel súlyozva számítja.
    """
    print("\\n--- Diagnosztika es vizualizacio generalasa (súlyozott metrikákkal) ---")
    os.makedirs(cfg['output_dir'], exist_ok=True)
    
    diagnostics_list = []
    max_violation_overall = 0.0
    epsilon = 1e-12 # Numerikus stabilitáshoz

    for i, point_data in enumerate(all_portfolios):
        # Alap metrikák (ezek a run_single_optimization_py-ból jönnek, már súlyozottak)
        metrics = {
            "Portfolio_ID": i + 1,
            "Evesitett_Hozam": point_data['achieved_wealth']**(12 / cfg['num_months']) - 1,
            "Evesitett_Kockazat_cCVaR": point_data['terminal_cCVaR'] / np.sqrt(cfg['num_months'] / 12)
        }

        weights = point_data['final_weights']
        max_violation_for_portfolio = 0.0
        
        # Lineáris kényszerek ellenőrzése (ez nem változik)
        for t in range(cfg['num_months']):
            w_t = weights[t, :]
            cs = constraints_strict if t < 24 else (constraints_laza if t < 48 else constraints_nagyon_laza)
            ineq_violations = np.maximum(0, -( (w_t @ cs['A_ineq'].T) - cs['b_ineq']) )
            eq_violations = np.abs( (w_t @ cs['A_eq'].T) - cs['b_eq'] )
            current_max_violation = max(np.max(ineq_violations), np.max(eq_violations))
            max_violation_for_portfolio = max(max_violation_for_portfolio, current_max_violation)

        metrics["Maximalis_Kenyszer_Sertes"] = max_violation_for_portfolio
        max_violation_overall = max(max_violation_overall, max_violation_for_portfolio)
        
        # --- Reszletes palya-metrikak SÚLYOZOTT szamitasa ---
        with torch.no_grad():
            w_gpu = torch.tensor(point_data['final_weights'], dtype=torch.float32, device=DEVICE)
            probs_gpu = torch.tensor(probabilities, dtype=torch.float32, device=DEVICE)
            tx_gpu = torch.tensor(etf_meta['tx_cost'].values, dtype=torch.float32, device=DEVICE).unsqueeze(1)
            w_lagged = torch.cat([w_gpu[0:1,:], w_gpu[:-1,:]], dim=0)
            
            turnover = 0.5 * torch.sum(torch.abs(w_gpu - w_lagged), dim=1)
            tx_cost = 0.5 * torch.sum(torch.abs(w_gpu - w_lagged) * tx_gpu.T, dim=1)
            net_ret = torch.einsum("mas,ma->ms", scenarios_tensor, w_gpu) - tx_cost.unsqueeze(1)
            
            # Súlyozott havi kockázatok
            q_level_tensor = torch.tensor(0.05, device=DEVICE)
            exp_ret = torch.sum(net_ret * probs_gpu.unsqueeze(0), dim=1, keepdim=True)
            
            var_thr = torch.stack([
                weighted_quantile(net_ret[t], q_level_tensor, probs_gpu) for t in range(net_ret.shape[0])
            ]).unsqueeze(1)
            
            monthly_tail_mask = net_ret < var_thr
            tail_returns_monthly = torch.where(monthly_tail_mask, net_ret, 0.0)
            tail_probs_monthly = torch.where(monthly_tail_mask, probs_gpu.unsqueeze(0), 0.0)
            
            sum_weighted_tail_returns = torch.sum(tail_returns_monthly * probs_gpu.unsqueeze(0), dim=1, keepdim=True)
            sum_monthly_tail_probs = torch.sum(tail_probs_monthly, dim=1, keepdim=True)
            
            cvar_raw = sum_weighted_tail_returns / (sum_monthly_tail_probs + epsilon)
            monthly_cCVaRs_tensor = -1 * (cvar_raw - exp_ret)
            
            hhi = np.sum(point_data['final_weights']**2, axis=1)
            normalized_hhi = (hhi - 1/w_gpu.shape[1]) / (1 - 1/w_gpu.shape[1])
            
            metrics["Atlagos_Havi_Forgas"] = torch.mean(turnover).item()
            metrics["Atlagos_Havi_cCVaR"] = torch.mean(monthly_cCVaRs_tensor).item()
            metrics["Maximalis_Havi_cCVaR"] = torch.max(monthly_cCVaRs_tensor).item()
            metrics["Atlagos_Koncentracio_HHI"] = np.mean(normalized_hhi)
        
        diagnostics_list.append(metrics)

    diagnostics_df = pd.DataFrame(diagnostics_list)
    print(">>> Hibrid Front Diagnosztika:")
    print(diagnostics_df.to_markdown(index=False, floatfmt=(".4f"))) # Markdown formázás egyszerűsítve

    print(f"\\n>>> ELLENŐRZÉS: A legnagyobb kényszersértés az összes portfólióban: {max_violation_overall:.4E}")
    if max_violation_overall > 1e-6:
        print("!!! FIGYELMEZTETÉS: Jelentős kényszersértés detektálva!")
    else:
        print(">>> A lineáris kényszerek mindenhol teljesülnek a megadott tolerancián belül.")

    # --- 2. Konvergencia abrak ---
    print("\n>>> Konvergencia abrak generalasa...")
    for i, point_data in enumerate(all_portfolios):
        if not point_data['loss_history'].empty:
            loss_df = point_data['loss_history'].melt(id_vars='Epoch', var_name='Veszteseg_Tipus', value_name='Ertek')
            fig = px.line(loss_df, x='Epoch', y='Ertek', color='Veszteseg_Tipus',
                          facet_col='Veszteseg_Tipus', facet_col_wrap=4,
                          title=f'Konvergencia: Portfolio Pont {i+1}', labels={'Ertek': 'Veszteseg Erteke'})
            fig.update_yaxes(matches=None)
            fig.show()
            fig.write_image(os.path.join(cfg['output_dir'], f'konvergencia_p{i+1}.png'))

    # --- 3. Hatekonysagi front abra ---
    print("\n>>> Hatekonysagi front abra generalasa...")
    efficient_path_df = diagnostics_df.sort_values('Evesitett_Kockazat_cCVaR').copy()
    efficient_path_df['max_hozam_eddig'] = efficient_path_df['Evesitett_Hozam'].cummax()
    efficient_path_df = efficient_path_df[efficient_path_df['Evesitett_Hozam'] >= efficient_path_df['max_hozam_eddig']]
    
    fig_front = go.Figure()
    fig_front.add_trace(go.Scatter(
        x=diagnostics_df['Evesitett_Kockazat_cCVaR'], y=diagnostics_df['Evesitett_Hozam'],
        mode='markers+text', text=diagnostics_df['Portfolio_ID'], textposition="top center",
        marker=dict(color='red', size=10, opacity=0.6), name='Portfolio Pontok'
    ))
    fig_front.add_trace(go.Scatter(
        x=efficient_path_df['Evesitett_Kockazat_cCVaR'], y=efficient_path_df['Evesitett_Hozam'],
        mode='lines', line=dict(color='red', width=2), name='Hatekony Front'
    ))
    fig_front.update_layout(title='Hatekonysagi Front', xaxis_title='Evesitett Kockazat (cCVaR)',
                            yaxis_title='Evesitett Hozam', xaxis_tickformat='.0%', yaxis_tickformat='.0%',
                            showlegend=False, template='plotly_white')
    fig_front.show()
    fig_front.write_image(os.path.join(cfg['output_dir'], 'hatekony_front.png'))

    # --- 4. Allokacios abrak ---
    print("\n>>> Allokacios abrak generalasa...")
    all_weights_list = []
    for i, point_data in enumerate(all_portfolios):
        df = pd.DataFrame(point_data['final_weights'], columns=etf_meta['symbol'])
        df['Honap'] = range(1, len(df) + 1)
        df['Portfolio'] = f'P{i+1}'
        all_weights_list.append(df)
    
    plot_data_all = pd.concat(all_weights_list, ignore_index=True)
    plot_data_melted = plot_data_all.melt(id_vars=['Honap', 'Portfolio'], var_name='symbol', value_name='Suly')
    plot_data_merged = pd.merge(plot_data_melted, etf_meta[['symbol', 'name']], on='symbol')
    
    fig_alloc = px.area(plot_data_merged, x='Honap', y='Suly', color='name',
                        facet_col='Portfolio', facet_col_wrap=4,
                        title='A front portfolioinak sulystrukturaja',
                        labels={'Suly': 'Suly', 'name': 'Eszkoz'})
    fig_alloc.update_yaxes(tickformat='.0%')
    fig_alloc.update_layout(legend=dict(orientation="h", yanchor="bottom", y=-0.2, xanchor="right", x=1))
    fig_alloc.show()
    fig_alloc.write_image(os.path.join(cfg['output_dir'], 'allokaciok.png'), width=1200, height=800)


def save_results_py(all_portfolios, scenarios_tensor, probabilities, etf_meta, cfg):
    """
    Elmenti az optimalizalas eredmenyeit (CSV, HDF5).
    MODOSÍTVA: A posterior valószínűségeket is elmenti a HDF5 fájlba
    a teljes reprodukálhatóság érdekében.
    """
    print("\\n--- Eredmenyek mentese ---\\")
    os.makedirs(cfg['output_dir'], exist_ok=True)
    
    # Mentes HDF5 formatumba
    hdf5_path = os.path.join(cfg['output_dir'], "optimization_results.h5")
    if os.path.exists(hdf5_path): os.remove(hdf5_path)
    
    with h5py.File(hdf5_path, 'w') as f:
        print(f">>> Eredmények mentése a(z) '{hdf5_path}' fájlba...")
        
        # --- ÚJ: A valószínűségek mentése ---
        # A betöltő függvény a 'posterior_probabilities' nevet keresi.
        f.create_dataset('posterior_probabilities', data=probabilities)
        print(">>> A posterior valószínűségek elmentve.")
        
        # A szcenáriók mentése R-kompatibilis formátumba (opcionális, de jó gyakorlat)
        # Ez a transzponálás a Python (idő, eszköz, szcenárió) -> (szcenárió, eszköz, idő) sorrendet hozza létre
        # Győződj meg róla, hogy ez a kívánt "kanonikus" forma.
        scenarios_to_save = np.transpose(scenarios_tensor.cpu().numpy(), (2, 1, 0))
        f.create_dataset('/scenarios', data=scenarios_to_save)
        print(">>> A szcenáriók elmentve.")
        
        # Az egyes optimalizált pontok adatainak mentése
        for i, point_data in enumerate(all_portfolios):
            group_name = f'/point_{i+1:02d}'
            grp = f.create_group(group_name)
            grp.create_dataset('weights', data=point_data['final_weights'])
            
            # A loss history DataFrame-et string formátumban kell menteni, vagy komponensenként
            loss_df = point_data['loss_history']
            grp.create_dataset('loss_history_values', data=loss_df.to_numpy())
            # Az oszlopneveket és indexet attribútumként mentjük
            grp['loss_history_values'].attrs['columns'] = list(loss_df.columns)

            grp.create_dataset('achieved_wealth', data=point_data['achieved_wealth'])
            grp.create_dataset('terminal_cCVaR', data=point_data['terminal_cCVaR'])
            
            # Mentes CSV formatumba (változatlan)
            weights_df = pd.DataFrame(point_data['final_weights'], columns=etf_meta['symbol'])
            weights_df.to_csv(os.path.join(cfg['output_dir'], f"{cfg['output_prefix']}_{i+1:02d}_weights.csv"))
            loss_df.to_csv(os.path.join(cfg['output_dir'], f"{cfg['output_prefix']}_{i+1:02d}_loss_history.csv"), index=False)

    print(">>> Az osszes portfolio adatai sikeresen elmentve CSV es HDF5 formatumban.")

>>> Szukseges csomagok telepitese...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.1/220.1 kB 16.0 MB/s eta 0:00:00
>>> Telepites befejezve.
>>> Hasznalt eszkoz: cuda


In [2]:
### A teljes fő munkafolyamat az új adatbetöltési logikával

if __name__ == '__main__':
    
    DATA_PATH = "/kaggle/input/download-etf-macro-data-in-python/financial_factors.csv.gz"
    META_PATH = "/kaggle/input/download-etf-macro-data-in-python/financial_factors_meta.csv.gz"
    
    config['scenario_cache'] = "/kaggle/input/bvar-sv-entropy-pooling/ep_posterior_for_python.h5"

    etf_meta_df, hist_returns = load_prepared_data_from_csv(DATA_PATH, META_PATH)
    
    # --- MÓDOSÍTÁS: Kinyerjük a befektethető szimbólumokat a metaadatokból ---
    investable_symbols = etf_meta_df['symbol'].tolist()
    
    # --- MÓDOSÍTÁS: Átadjuk a szimbólumokat a betöltő függvénynek ---
    # A függvény már a leszűrt, 11 eszközös tömböt adja vissza
    scenarios, probabilities = generate_or_load_scenarios_py(hist_returns, config, investable_symbols)
    
    # A tenzorrá alakítás a helyes, 11-es dimenzióval fog történni
    scenarios_tensor = torch.tensor(scenarios, dtype=torch.float32).to(DEVICE)
    
    # --- A KÓD TÖBBI RÉSZE VÁLTOZATLAN ---
    # A bázisportfóliók, a kényszerek és az optimalizáció már eleve
    # a 11 befektethető eszközre lettek felépítve, így innentől minden konzisztens.
    
    constraints_spec_strict =     {'min_treasury': 0.50, 'max_gld': 0.25, 'min_gld': 0.10, 'min_bil_shy': 0.10}
    constraints_spec_laza =       {'min_treasury': 0.45, 'max_gld': 0.30, 'min_gld': 0.08, 'min_bil_shy': 0.08}
    constraints_spec_nagyon_laza ={'min_treasury': 0.40, 'max_gld': 0.35, 'min_gld': 0.05, 'min_bil_shy': 0.05}

    constraints_strict = build_constraints_from_dataframe(investable_symbols, constraints_spec_strict)
    constraints_laza = build_constraints_from_dataframe(investable_symbols, constraints_spec_laza)
    constraints_nagyon_laza = build_constraints_from_dataframe(investable_symbols, constraints_spec_nagyon_laza)

    basis_strict_np = generate_cvar_frontier_portfolios_py(hist_returns, constraints_strict, num_points=10)
    basis_laza_np = generate_cvar_frontier_portfolios_py(hist_returns, constraints_laza, num_points=10)
    basis_nagyon_laza_np = generate_cvar_frontier_portfolios_py(hist_returns, constraints_nagyon_laza, num_points=10)
    
    basis_strict_t = torch.tensor(basis_strict_np, dtype=torch.float32).to(DEVICE)
    basis_laza_t = torch.tensor(basis_laza_np, dtype=torch.float32).to(DEVICE)
    basis_nagyon_laza_t = torch.tensor(basis_nagyon_laza_np, dtype=torch.float32).to(DEVICE)

    print("\\n--- A HATEKONYSAGI FRONT FELTERKEPEZESE ---")
    
    print("\\n'Min. kockázatu' jellegű portfolio szamitasa...")
    point_min_risk = run_single_optimization_py(1.2, basis_strict_t, basis_laza_t, basis_nagyon_laza_t, 
                                                etf_meta_df, scenarios_tensor, probabilities, config)

    print("\\n'Max. hozamu' jellegű portfolio szamitasa...")
    point_max_wealth = run_single_optimization_py(2.0, basis_strict_t, basis_laza_t, basis_nagyon_laza_t, 
                                                  etf_meta_df, scenarios_tensor, probabilities, config)

    target_wealth_grid = np.linspace(point_min_risk['achieved_wealth'], point_max_wealth['achieved_wealth'], 8)

    intermediate_points = []
    for i, target in enumerate(target_wealth_grid):
        print(f"\\nFrontier pont {i+1}/8 szamitasa (Celvagyon: {target:.3f})...")
        point = run_single_optimization_py(target, basis_strict_t, basis_laza_t, basis_nagyon_laza_t, 
                                           etf_meta_df, scenarios_tensor, probabilities, config)
        intermediate_points.append(point)

    all_frontier_portfolios = [point_min_risk] + intermediate_points + [point_max_wealth]

    save_results_py(all_frontier_portfolios, scenarios_tensor, probabilities, etf_meta_df, config)
    
    generate_diagnostics_py(all_frontier_portfolios, etf_meta_df, scenarios_tensor, probabilities, config, 
                            constraints_strict, constraints_laza, constraints_nagyon_laza)

    print('\\n>>> A Python szkript futasa befejezodott. <<<')

>>> Adatok betöltése a központosított CSV forrásból...
Betöltve 220 hónapnyi adat 11 eszközre.
Toltott szcenariok a(z) '/kaggle/input/bvar-sv-entropy-pooling/ep_posterior_for_python.h5' cache-bol...
Várt dimenzióméretek: Idő=60, Eszközök=14, Szcenáriók=5000
Beolvasott tömb mérete: (5000, 14, 60)
Helyes sorrendbe transzponált teljes tömb mérete: (60, 14, 5000)
Szűrés utáni szcenárió-tömb mérete: (60, 11, 5000)

--- CVaR Hatékony Front generálása (10 ponttal) ---
Front hozam tartománya: [0.0023, 0.0062] (havi hozam)
Számítás... Célhozam: 0.0062
>>> A bázisportfóliók generálása befejeződött.               

--- CVaR Hatékony Front generálása (10 ponttal) ---
Front hozam tartománya: [0.0022, 0.0065] (havi hozam)
Számítás... Célhozam: 0.0065
>>> A bázisportfóliók generálása befejeződött.               

--- CVaR Hatékony Front generálása (10 ponttal) ---
Front hozam tartománya: [0.0019, 0.0068] (havi hozam)
Számítás... Célhozam: 0.0068
>>> A bázisportfóliók generálása befejeződött.         

/tmp/ipykernel_19/1966606060.py:338: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  q_level_tensor = torch.tensor(0.05, device=DEVICE)
/tmp/ipykernel_19/1966606060.py:292: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  if not torch.isclose(torch.sum(weights), torch.tensor(1.0)):
/tmp/ipykernel_19/1966606060.py:292: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be tre

Epoch 500/10000, Loss: 0.1838, LR: 0.01000
Epoch 1000/10000, Loss: 0.1820, LR: 0.01000
Epoch 1500/10000, Loss: 0.1812, LR: 0.01000
Epoch 2000/10000, Loss: 0.1806, LR: 0.01000
Epoch 2500/10000, Loss: 0.1793, LR: 0.01000
Epoch 3000/10000, Loss: 0.1790, LR: 0.01000
Epoch 3500/10000, Loss: 0.1784, LR: 0.01000
Epoch 4000/10000, Loss: 0.1782, LR: 0.00200
Epoch 4500/10000, Loss: 0.1777, LR: 0.00200
Epoch 5000/10000, Loss: 0.1776, LR: 0.00200
Epoch 5500/10000, Loss: 0.1774, LR: 0.00200
Epoch 6000/10000, Loss: 0.1772, LR: 0.00200
Epoch 6500/10000, Loss: 0.1773, LR: 0.00200
Epoch 7000/10000, Loss: 0.1768, LR: 0.00200
Epoch 7500/10000, Loss: 0.1766, LR: 0.00200
Epoch 8000/10000, Loss: 0.1764, LR: 0.00040
Epoch 8500/10000, Loss: 0.1764, LR: 0.00040
Epoch 9000/10000, Loss: 0.1763, LR: 0.00040
Epoch 9500/10000, Loss: 0.1763, LR: 0.00040
Epoch 10000/10000, Loss: 0.1762, LR: 0.00040
\n'Max. hozamu' jellegű portfolio szamitasa...
Epoch 500/10000, Loss: 3918.5527, LR: 0.01000
Epoch 1000/10000, Loss: 387


>>> Hatekonysagi front abra generalasa...



>>> Allokacios abrak generalasa...


\n>>> A Python szkript futasa befejezodott. <<<
